In [2]:
try:
  %tensorflow_version 2.x
except Exception:
  print("exception")
  pass
import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

np.set_printoptions(precision=4)

In [0]:
dataset = tf.data.Dataset.from_tensor_slices(['0'])
# dataset.map(lambda x:x+1)

In [0]:
dataset.element_spec

TensorSpec(shape=(), dtype=tf.string, name=None)

In [0]:
for elem in dataset:
  print(elem.numpy())

b'0'


In [0]:
tf.random.uniform([4, 10], minval=1, maxval=10)

<tf.Tensor: id=15, shape=(4, 10), dtype=float32, numpy=
array([[9.9679, 6.6588, 1.6417, 7.9961, 2.4157, 3.4829, 5.019 , 7.7037,
        8.095 , 2.2067],
       [5.0784, 2.8311, 1.4909, 2.6102, 4.1314, 7.1442, 7.2381, 5.8384,
        1.0182, 2.4741],
       [3.3935, 8.0384, 5.8379, 8.1665, 4.4845, 2.3247, 1.3426, 3.0278,
        7.896 , 5.2765],
       [1.5554, 3.5266, 3.1841, 1.7964, 9.9613, 8.0443, 9.1335, 6.1555,
        6.3804, 7.8233]], dtype=float32)>

In [0]:
dataset1 = tf.data.Dataset.from_tensor_slices(
    tf.random.uniform([4, 10], minval=1, maxval=10, dtype=tf.int32))

dataset1

<TensorSliceDataset shapes: (10,), types: tf.int32>

In [0]:
for z in dataset1:
  print(z.numpy())

[5 9 6 5 6 2 3 8 4 5]
[8 2 3 8 5 8 1 2 8 9]
[7 4 8 5 9 9 1 7 3 1]
[5 5 8 7 5 3 5 5 3 5]


In [0]:
(x_train, y_train), (x_test, y_test) =tf.keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
images = x_train/255
labels = y_train

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((images,labels))
dataset

<TensorSliceDataset shapes: ((28, 28), ()), types: (tf.float64, tf.uint8)>

Python Image generator:

In [0]:
flower = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

228818944/228813984 [==============================] - 4s 0us/step


In [0]:
data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255, rotation_range=25)

In [0]:
images, labels = next(data_gen.flow_from_directory(flower))

Found 3670 images belonging to 5 classes.


In [0]:
for x,y in data_gen.flow_from_directory(flower):
  print(y)
  break

In [0]:
print(images.dtype, images.shape)
print(labels.dtype, labels.shape)

float32 (32, 256, 256, 3)
float32 (32, 5)


In [4]:
data = tf.data.Dataset.from_generator(data_gen.flow_from_directory, args=flower, output_types=(tf.float32, tf.float32), output_shapes=([32,256,256,3],[32,5]))

NameError: ignored

In [5]:
titanic_dataset_path = tf.keras.utils.get_file("train.csv", "https://storage.googleapis.com/tf-datasets/titanic/train.csv")

32768/30874 [===============================] - 0s 0us/step


In [0]:
df = pd.read_csv(titanic_dataset_path,index_col=False)

In [7]:
df.head()

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [8]:
titanic_slices = tf.data.Dataset.from_tensor_slices(dict(df))

for feature_batch in titanic_slices.take(1):
  for key, value in feature_batch.items():
    print("  {!r:20s}: {}".format(key, value))

  'survived'          : 0
  'sex'               : b'male'
  'age'               : 22.0
  'n_siblings_spouses': 1
  'parch'             : 0
  'fare'              : 7.25
  'class'             : b'Third'
  'deck'              : b'unknown'
  'embark_town'       : b'Southampton'
  'alone'             : b'n'


In [9]:
titanic_batches = tf.data.experimental.make_csv_dataset(titanic_dataset_path, batch_size=4, label_name='survived')

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.
Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizations will take care of using the fused implementation.


In [10]:
for feature_batch, label_batch in titanic_batches.take(1):
  print("'survived': {}".format(label_batch))
  print("features:")
  for key, value in feature_batch.items():
    print("  {!r:20s}: {}".format(key, value))

'survived': [0 0 1 1]
features:
  'sex'               : [b'male' b'female' b'female' b'female']
  'age'               : [36. 28. 22.  5.]
  'n_siblings_spouses': [0 3 0 0]
  'parch'             : [0 1 1 0]
  'fare'              : [12.875  25.4667 55.     12.475 ]
  'class'             : [b'Second' b'Third' b'First' b'Third']
  'deck'              : [b'D' b'unknown' b'E' b'unknown']
  'embark_town'       : [b'Cherbourg' b'Southampton' b'Southampton' b'Southampton']
  'alone'             : [b'y' b'n' b'n' b'y']


In [11]:
inc_dataset = tf.data.Dataset.range(100)
dec_dataset = tf.data.Dataset.range(0, -100, -1)
dataset = tf.data.Dataset.zip((inc_dataset, dec_dataset))
batched_dataset = dataset.batch(4)

it = iter(batched_dataset)
for batch in batched_dataset.take(4):
  print([arr.numpy() for arr in batch])

[array([0, 1, 2, 3]), array([ 0, -1, -2, -3])]
[array([4, 5, 6, 7]), array([-4, -5, -6, -7])]
[array([ 8,  9, 10, 11]), array([ -8,  -9, -10, -11])]
[array([12, 13, 14, 15]), array([-12, -13, -14, -15])]


Using High-level APIs:

In [12]:
train, test = tf.keras.datasets.fashion_mnist.load_data()

images, labels = train
images = images/255.0
labels = labels.astype(np.int32)

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
fmnist_train_ds = tf.data.Dataset.from_tensor_slices((images, labels))
fmnist_train_ds = fmnist_train_ds.shuffle(5000).batch(32)

In [0]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
              metrics=['accuracy'])

In [15]:
model.fit(fmnist_train_ds, epochs=2)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/2
1875/1875 [==============================] - 11s 6ms/step - loss: 0.6065 - accuracy: 0.7950
Epoch 2/2
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4630 - accuracy: 0.8425


In [16]:
loss, accuracy = model.evaluate(fmnist_train_ds)
print("Loss :", loss)
print("Accuracy :", accuracy)

1875/1875 [==============================] - 4s 2ms/step - loss: 0.4516 - accuracy: 0.8450
Loss : 0.45156649941603344
Accuracy : 0.84503335


In [17]:
model.fit(fmnist_train_ds.repeat(), epochs=2, steps_per_epoch=20)

Train for 20 steps
Epoch 1/2
20/20 [==============================] - 0s 8ms/step - loss: 0.4162 - accuracy: 0.8453
Epoch 2/2
20/20 [==============================] - 0s 3ms/step - loss: 0.5234 - accuracy: 0.8156


In [18]:
loss, accuracy = model.evaluate(fmnist_train_ds.repeat(), steps=10)
print("Loss :", loss)
print("Accuracy :", accuracy)

10/10 [==============================] - 0s 4ms/step - loss: 0.4213 - accuracy: 0.8531
Loss : 0.4213171511888504
Accuracy : 0.853125


In [19]:
predict_ds = tf.data.Dataset.from_tensor_slices(images).batch(32)
result = model.predict(predict_ds , steps = 10)
print(result.shape)

(320, 10)


In [20]:
result = model.predict(fmnist_train_ds, steps = 10)
print(result.shape)

(320, 10)


## make_csv_dataset:

Reads CSV files into a dataset, where each element is a (features, labels) tuple that corresponds to a batch of CSV rows. The features dictionary maps feature column names to Tensors containing the corresponding feature data, and labels is a Tensor containing the batch's label data.


### tf.estimator

To use a `Dataset` in the `input_fn` of a `tf.estimator.Estimator`, simply
return the `Dataset` from the `input_fn` and the framework will take care of consuming its elements
for you. For example:

In [0]:
import tensorflow_datasets as tfds

def input_train_fn():
  titanic = tf.data.experimental.make_csv_dataset(titanic_dataset_path, batch_size=5, label_name="survived")
  titanic_batches = (titanic.cache().repeat().shuffle(500))
  return titanic_batches

  

In [0]:
embark = tf.feature_column.categorical_column_with_hash_bucket('embark_town', 32)
cls = tf.feature_column.categorical_column_with_vocabulary_list('class',['First', 'Second', 'Third'])
age = tf.feature_column.numeric_column('age')

In [42]:

import tempfile

model_dir = tempfile.mkdtemp()
model = tf.estimator.LinearClassifier(model_dir=model_dir, feature_columns=[embark, cls, age], n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpin044lwy', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f56dfcf39b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [43]:
model = model.train(input_fn=input_train_fn, steps=100)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpin044lwy/model.ckpt.
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:Saving checkpoints for 100 into /tmp/tmpin044lwy/model.ckpt.
INFO:tensorflow:Loss for final step: 0.5311256.


In [45]:
result = model.evaluate(input_train_fn, steps=10)

for key, value in result.items():
  print(key, ":", value)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-11-05T07:14:52Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpin044lwy/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/10]
INFO:tensorflow:Evaluation [2/10]
INFO:tensorflow:Evaluation [3/10]
INFO:tensorflow:Evaluation [4/10]
INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2019-11-05-07:14:53
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.82, accuracy_baseline = 0.72, auc = 0.70436513, auc_precision_recall = 0.60081613, average_loss = 0.5484646, global_step = 100, label/mean = 0.28, loss = 0.5484646, precision = 0.85714287, prediction/mean = 0.36

In [46]:
for pred in model.predict(input_train_fn):
  for key, value in pred.items():
    print(key, ":", value)
  break

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpin044lwy/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
logits : [-1.1225]
logistic : [0.2455]
probabilities : [0.7545 0.2455]
class_ids : [0]
classes : [b'0']
all_class_ids : [0 1]
all_classes : [b'0' b'1']
